In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [28]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle
import lightgbm as lgbm
import catboost as cb


from sklearn.model_selection import KFold
from itertools import product
from catboost import CatBoostRegressor


In [3]:
gc.collect()

items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)


In [4]:
sales_train = sales_train[sales_train['year'].isin([2013]) == False]

sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [5]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
#train_item_ids = ids_keep
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

#all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_item_ids = test_item_ids

#all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))
all_shop_ids = test_shop_ids



In [6]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [7]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [8]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [9]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [10]:
mean_prices = sales_train.groupby(['item_id','date_block_num'])['item_price'].mean().reset_index()
training = training.merge(mean_prices, on=['item_id','date_block_num'], how='left')

In [16]:
training["shop_cat"] = training["shop_id"].astype(str) + "_" + training["item_category_id"].astype(str)

In [17]:
#https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study

from sklearn.model_selection import StratifiedKFold
columns = ["item_id", "shop_id", "item_category_id", "shop_cat"]



y_train = training["shop_item_cnt_block"].values
folds = KFold(n_splits = 5, shuffle=True).split(training)

i=1
for in_fold_index, out_of_fold_index in folds:
    print("fold", i)
    #print(np.intersect1d(training.loc[in_fold_index]["shop_id"].unique(), training.loc[out_of_fold_index]["shop_id"].unique()))
    #print(len(in_fold_index))
    for column in columns:
        means = training.iloc[in_fold_index].groupby(column)['shop_item_cnt_block'].mean()
            #x_validation[column + "_mean_target"] = means\
        name = column + '_mean_encoding'
        training.loc[out_of_fold_index,name] = training.loc[out_of_fold_index][column].map(means)
    i+=1

fold 1
fold 2
fold 3
fold 4
fold 5


In [18]:
training['item_cnt_block_mean'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.mean)
#training['item_cnt_block_min'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.min)
#training['item_cnt_block_max'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.max)
#training['item_cnt_block_std'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.std)
#training['item_cnt_block_med'] = training.groupby(['date_block_num'])['item_cnt_block'].transform(np.median)

training['shop_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.mean)
#training['shop_cnt_block_min'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.min)
#training['shop_cnt_block_max'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.max)
#training['shop_cnt_block_std'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.std)
#training['shop_cnt_block_med'] = training.groupby(['date_block_num'])['shop_cnt_block'].transform(np.median)

training['category_cnt_block_mean'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.mean)
#training['category_cnt_block_min'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.min)
#training['category_cnt_block_max'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.max)
#training['category_cnt_block_std'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.std)
#training['category_cnt_block_med'] = training.groupby(['date_block_num'])['category_cnt_block'].transform(np.median)

training['shop_category_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.mean)
#training['shop_category_cnt_block_min'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.min)
#training['shop_category_cnt_block_max'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.max)
#training['shop_category_cnt_block_std'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.std)
#training['shop_category_cnt_block_med'] = training.groupby(['date_block_num'])['shop_category_cnt_block'].transform(np.median)

#training['shop_item_cnt_block_mean'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.mean)
#training['shop_item_cnt_block_min'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.min)
#training['shop_item_cnt_block_max'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.max)
#training['shop_item_cnt_block_std'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.std)
#training['shop_item_cnt_block_med'] = training.groupby(['date_block_num'])['shop_item_cnt_block'].transform(np.median)

In [19]:
training['comp1'] = training['item_cnt_block'] * training['shop_cnt_block']
training['comp2'] = training['item_cnt_block'] * training['category_cnt_block']
training['comp3'] = training['item_cnt_block'] * training['shop_category_cnt_block']

training['comp4'] = training['item_cnt_block'] * training['item_id_mean_encoding']
training['comp5'] = training['item_cnt_block'] * training['shop_id_mean_encoding']
training['comp6'] = training['item_cnt_block'] * training['shop_cat_mean_encoding']

training['comp7'] = training['shop_cnt_block'] * training['shop_id_mean_encoding']
training['comp8'] = training['shop_cnt_block'] * training['shop_cat_mean_encoding']

training['comp9'] = training['shop_category_cnt_block'] * training['shop_id_mean_encoding']
training['comp10'] = training['shop_category_cnt_block'] * training['shop_cat_mean_encoding']

training['comp11'] = training['item_cnt_block_mean'] * training['shop_cnt_block_mean']
training['comp12'] = training['item_cnt_block_mean'] * training['category_cnt_block_mean']

training['comp13'] = training['item_cnt_block_mean'] * training['shop_category_cnt_block_mean']
training['comp14'] = training['item_cnt_block'] * training['shop_category_cnt_block_mean']

In [20]:
training.fillna(0,inplace=True)

In [21]:
def add_lags(df, cols, name, lags = [1]):
    
    for lag in lags:
        print(name, lag)
        lag_name = name + "_lag_" + str(lag)
        
        try:
            df.drop(columns=[lag_name],inplace=True)
        except:
            pass       

        result = df\
            .drop_duplicates(cols)\
            .sort_values(cols)\
            .set_index(cols)\
            .groupby(cols[0:len(cols)-1],as_index=False)\
            [name].shift(lag)\
            .rename(columns={name:lag_name}).reset_index()

        df = df.merge(result, on=cols, how='left')
        df[lag_name].fillna(0,inplace=True)
        if "mean" in name:
            df[lag_name] = pd.to_numeric(df[lag_name], downcast='float')
        else:
            df[lag_name] = pd.to_numeric(df[lag_name].astype(int), downcast='unsigned')
        del result
        gc.collect()
    
    return df
                                         

                                        
training = add_lags(training, ['item_id','date_block_num'], 'item_cnt_block')
training = add_lags(training, ['item_id','date_block_num'], 'item_cnt_block_mean')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_cnt_block')
training = add_lags(training, ['shop_id','date_block_num'], 'shop_cnt_block_mean')
training = add_lags(training, ['item_category_id','date_block_num'], 'category_cnt_block')
training = add_lags(training, ['item_category_id','date_block_num'], 'category_cnt_block_mean')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_category_cnt_block')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'shop_category_cnt_block_mean')

item_cnt_block 1
item_cnt_block_mean 1
shop_cnt_block 1
shop_cnt_block_mean 1
category_cnt_block 1
category_cnt_block_mean 1
shop_category_cnt_block 1
shop_category_cnt_block_mean 1


In [22]:
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'comp1')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'comp2')

training = add_lags(training, ['shop_id','item_id','date_block_num'], 'comp3')
training = add_lags(training, ['item_id','date_block_num'], 'comp4')

training = add_lags(training, ['shop_id','item_id','date_block_num'], 'comp5')
training = add_lags(training, ['shop_id','item_id','date_block_num'], 'comp6')

training = add_lags(training, ['shop_id','date_block_num'], 'comp7')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'comp8')

comp1 1
comp2 1
comp3 1
comp4 1
comp5 1
comp6 1
comp7 1
comp8 1


In [23]:
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'comp9')
training = add_lags(training, ['shop_id','item_category_id','date_block_num'], 'comp10')


training = add_lags(training, ['shop_id','item_id','date_block_num'], 'comp11')
training = add_lags(training, ['item_id','date_block_num'], 'comp12')

training = add_lags(training, ['item_id','shop_id','date_block_num'], 'comp13')
training = add_lags(training, ['item_id','shop_id','date_block_num'], 'comp14')

comp9 1
comp10 1
comp11 1
comp12 1
comp13 1
comp14 1


In [42]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'item_price', 'item_cnt_block_mean', 'shop_cnt_block_mean',
       'category_cnt_block_mean', 'shop_category_cnt_block_mean', 'comp1',
       'comp2', 'comp3', 'item_id_mean_encoding', 'shop_id_mean_encoding',
       'item_category_id_mean_encoding', 'shop_cat',
       'shop_cat_mean_encoding', 'comp4', 'comp5', 'comp6', 'comp7',
       'comp8', 'comp9', 'comp10', 'comp11', 'comp12', 'comp13', 'comp14',
       'item_cnt_block_lag_1', 'item_cnt_block_mean_lag_1',
       'shop_cnt_block_lag_1', 'shop_cnt_block_mean_lag_1',
       'category_cnt_block_lag_1', 'category_cnt_block_mean_lag_1',
       'shop_category_cnt_block_lag_1',
       'shop_category_cnt_block_mean_lag_1', 'comp1_lag_1', 'comp2_lag_1',
       'comp3_lag_1', 'comp4_lag_1', 'comp5_lag_1', 'comp6_lag_1',
       'co

In [25]:
training.fillna(0,inplace=True)

In [69]:
features = [
  
  # 'item_id_mean_encoding',
   #    'shop_id_mean_encoding', 'item_category_id_mean_encoding',
    #   'shop_cat_mean_encoding',
  'item_cnt_block_lag_1',
       'item_cnt_block_lag_2', 'item_cnt_block_lag_3',
       'item_cnt_block_mean_lag_1', 'item_cnt_block_mean_lag_2',
       'item_cnt_block_mean_lag_3', 'shop_cnt_block_lag_1',
       'shop_cnt_block_lag_2', 'shop_cnt_block_lag_3',
       'shop_cnt_block_mean_lag_1', 'shop_cnt_block_mean_lag_2',
       'shop_cnt_block_mean_lag_3', 'category_cnt_block_lag_1',
       'category_cnt_block_lag_2', 'category_cnt_block_lag_3',
       'category_cnt_block_mean_lag_1', 'category_cnt_block_mean_lag_2',
       'category_cnt_block_mean_lag_3', 'shop_category_cnt_block_lag_1',
       'shop_category_cnt_block_lag_2', 'shop_category_cnt_block_lag_3',
       'shop_category_cnt_block_mean_lag_1',
       'shop_category_cnt_block_mean_lag_2',
       'shop_category_cnt_block_mean_lag_3', 'comp1_lag_1', 'comp1_lag_2',
       'comp1_lag_3', 'comp2_lag_1', 'comp2_lag_2', 'comp2_lag_3',
       'comp3_lag_1', 'comp3_lag_2', 'comp3_lag_3', 'comp4_lag_1',
       'comp4_lag_2', 'comp4_lag_3', 'comp5_lag_1', 'comp5_lag_2',
       'comp5_lag_3', 'comp6_lag_1', 'comp6_lag_2', 'comp6_lag_3',
       'comp7_lag_1', 'comp7_lag_2', 'comp7_lag_3', 'comp8_lag_1',
       'comp8_lag_2', 'comp8_lag_3', 'comp9_lag_1', 'comp9_lag_2',
       'comp9_lag_3', 'comp10_lag_1', 'comp10_lag_2', 'comp10_lag_3',
       'comp11_lag_1', 'comp11_lag_2', 'comp11_lag_3', 'comp12_lag_1',
       'comp12_lag_2', 'comp12_lag_3', 'comp13_lag_1', 'comp13_lag_2',
       'comp13_lag_3', 'comp14_lag_1', 'comp14_lag_2', 'comp14_lag_3'
     
]

In [22]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_price,shop_cat,item_id_mean_encoding,shop_id_mean_encoding,item_category_id_mean_encoding,shop_cat_mean_encoding,item_cnt_block_mean,shop_cnt_block_mean,category_cnt_block_mean,shop_category_cnt_block_mean,comp1,comp2,comp3,comp4,comp5,comp6,comp7,comp8,comp9,comp10,comp11,comp12,comp13,comp14,item_cnt_block_lag_1,item_cnt_block_lag_2,item_cnt_block_lag_3,item_cnt_block_mean_lag_1,item_cnt_block_mean_lag_2,item_cnt_block_mean_lag_3,shop_cnt_block_lag_1,shop_cnt_block_lag_2,shop_cnt_block_lag_3,shop_cnt_block_mean_lag_1,shop_cnt_block_mean_lag_2,shop_cnt_block_mean_lag_3,category_cnt_block_lag_1,category_cnt_block_lag_2,category_cnt_block_lag_3,category_cnt_block_mean_lag_1,category_cnt_block_mean_lag_2,category_cnt_block_mean_lag_3,shop_category_cnt_block_lag_1,shop_category_cnt_block_lag_2,shop_category_cnt_block_lag_3,shop_category_cnt_block_mean_lag_1,shop_category_cnt_block_mean_lag_2,shop_category_cnt_block_mean_lag_3,comp1_lag_1,comp1_lag_2,comp1_lag_3,comp2_lag_1,comp2_lag_2,comp2_lag_3,comp3_lag_1,comp3_lag_2,comp3_lag_3,comp4_lag_1,comp4_lag_2,comp4_lag_3,comp5_lag_1,comp5_lag_2,comp5_lag_3,comp6_lag_1,comp6_lag_2,comp6_lag_3,comp7_lag_1,comp7_lag_2,comp7_lag_3,comp8_lag_1,comp8_lag_2,comp8_lag_3,comp9_lag_1,comp9_lag_2,comp9_lag_3,comp10_lag_1,comp10_lag_2,comp10_lag_3,comp11_lag_1,comp11_lag_2,comp11_lag_3,comp12_lag_1,comp12_lag_2,comp12_lag_3,comp13_lag_1,comp13_lag_2,comp13_lag_3,comp14_lag_1,comp14_lag_2,comp14_lag_3
1135879,5184,19,31,0,67,8,2015,1.0,1533.0,409.0,10.0,1099.000000,19_67,0.022576,0.233433,0.141483,0.116264,11.840196,1551.500000,3253.890392,75.921218,1533.0,409.0,10.0,0.022576,0.233433,0.116264,357.852731,178.232528,2.334330,1.162639,18370.064216,38526.700261,898.922113,75.921218,1,1,3,10.808824,10.833333,11.949804,1440,1300,1459,1427.642822,1430.904785,1592.166626,474,417,512,3335.517822,3318.272217,3428.631348,7,12,8,75.583504,74.266708,75.539597,1440,1300,4377,1422,3753,2048,7,12,24,0,0,0,0,0,0,0,0,0,338,307,342,165,162,167,1,2,1,0,1,0,15431,15501,19026,36053,35947,40971,816,804,902,75,74,226
4396263,20565,53,15,0,72,4,2014,0.0,1641.0,1850.0,37.0,0.000000,53_72,0.012146,0.212639,0.140252,0.102229,9.003333,1842.166667,5550.332157,106.891709,0.0,0.0,0.0,0.000000,0.000000,0.000000,348.940191,167.757504,7.867634,3.782467,16585.640556,49971.490519,962.381684,0.000000,0,0,0,11.849607,10.279804,10.266078,2059,1866,2144,2216.380859,2127.285645,2220.047607,1866,1860,1785,6547.391113,5804.443359,6563.625000,35,19,19,125.359169,112.041161,123.881836,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,437,386,445,212,189,221,7,3,3,3,1,1,26263,21868,22791,77584,59668,67382,1485,1151,1271,0,0,0
2771330,13249,18,24,0,47,1,2015,0.0,1337.0,0.0,0.0,0.000000,18_47,0.000000,0.186742,0.041803,0.017341,16.166078,2176.380952,5477.754314,107.258562,0.0,0.0,0.0,0.000000,0.000000,0.000000,249.674621,23.184971,0.000000,0.000000,35183.545173,88553.805863,1733.950327,0.000000,0,0,0,24.733137,16.610001,13.808432,1951,1498,1030,3318.500000,2298.428467,1926.547607,0,0,0,7494.178711,5516.103027,5176.736328,0,0,0,148.356445,106.721191,96.386688,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,361,279,191,35,27,18,0,0,0,0,0,0,82076,38176,26602,185354,91622,71482,3669,1772,1330,0,0,0
1746268,8249,55,30,0,38,7,2015,3.0,1658.0,702.0,0.0,199.000000,55_38,0.110505,0.155304,0.142760,0.000000,10.808824,1427.642857,3335.517843,75.583501,4974.0,2106.0,0.0,0.331514,0.465911,0.000000,257.493405,0.000000,0.000000,0.000000,15431.139706,36053.023746,816.968728,226.750504,1,4,5,10.833333,11.949804,12.535490,1909,2117,3422,1430.904785,1592.166626,1711.166626,640,926,1148,3318.272217,3428.631348,4010.790771,0,0,0,74.266708,75.539597,88.694931,1909,8468,17110,7040,19446,19516,0,0,0,0,0,0,0,0,0,0,0,0,291,326,521,0,0,0,0,0,0,0,0,0,15501,19026,21450,35947,40971,50277,804,902,1111,74,302,443
1080106,4964,57,28,1,20,5,2015,1

In [30]:
gc.collect()

ZEROS_KEEP=0.2


#x_train = training[(training['date_block_num'] < 33) & (training['val_ignore'] == False)]
x_train = training[(training['date_block_num'] < 33)]
y_train = x_train['shop_item_cnt_block']





x_val = training[training['date_block_num'] == 33]
y_val = x_val['shop_item_cnt_block']

#pos_val_len = len(y_val[y_val != 0])
#print("pos_val_len", pos_val_len)

#zeros_keep_indices_val = y_val[y_val == 0].sample(int(pos_val_len/ZEROS_KEEP)).index
#print("zeros_keep_indices_val", len(zeros_keep_indices_val))
#non_zeros_val_indices = y_val[y_val != 0].index
#print("non_zeros_val_indices", len(non_zeros_val_indices))

#val_indices = np.append(np.array(zeros_keep_indices_val), np.array(non_zeros_val_indices))

#y_val = y_val.loc[val_indices]
#x_val = x_val.loc[val_indices]

In [82]:
lgtrain = lgbm.Dataset(x_train[features], label=y_train)
lgval = lgbm.Dataset(x_val[features], label=y_val)



#[0.00542047893814942, 29, 24, 0.39949465609514856, 1, 0.67943500, 10]
params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "seed": 42,
        "learning_rate" : 0.05,
        "min_data_in_leaf": 10000,
        "num_leaves": 10,
        "max_depth" : 4,
        #"bagging_fraction": 0.7,
        #"bagging_freq": 1,
        #"feature_fraction": 0.7,
        #"lambda_l1": 10,
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 10000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=10, 
                      verbose_eval=10, 
                      evals_result=evals_result)


scores = {}
for i,score in enumerate(model_lgb.feature_importance()):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's rmse: 0.995522
[20]	valid_0's rmse: 0.957153
[30]	valid_0's rmse: 0.940218
[40]	valid_0's rmse: 0.930111
[50]	valid_0's rmse: 0.92395
[60]	valid_0's rmse: 0.919965
[70]	valid_0's rmse: 0.917632
[80]	valid_0's rmse: 0.91653
[90]	valid_0's rmse: 0.915139
[100]	valid_0's rmse: 0.914468
[110]	valid_0's rmse: 0.913343
[120]	valid_0's rmse: 0.912682
[130]	valid_0's rmse: 0.911556
[140]	valid_0's rmse: 0.910813
[150]	valid_0's rmse: 0.909541
[160]	valid_0's rmse: 0.908949
[170]	valid_0's rmse: 0.908262
[180]	valid_0's rmse: 0.907476
[190]	valid_0's rmse: 0.907028
[200]	valid_0's rmse: 0.906551
[210]	valid_0's rmse: 0.906068
[220]	valid_0's rmse: 0.905847
[230]	valid_0's rmse: 0.905359
[240]	valid_0's rmse: 0.905108
[250]	valid_0's rmse: 0.904915
[260]	valid_0's rmse: 0.904569
[270]	valid_0's rmse: 0.904217
[280]	valid_0's rmse: 0.903612
[290]	valid_0's rmse: 0.903149
[300]	valid_0's rmse: 0.902973
[310]	valid_0's rms

[('comp6_lag_1', 539),
 ('comp4_lag_1', 459),
 ('comp8_lag_1', 446),
 ('category_cnt_block_lag_1', 428),
 ('comp10_lag_1', 373),
 ('comp3_lag_1', 372),
 ('item_cnt_block_lag_1', 345),
 ('comp5_lag_1', 329),
 ('item_cnt_block_mean_lag_1', 295),
 ('comp14_lag_1', 202)]

In [43]:
features = [
     #'item_id','shop_id',
 'item_cnt_block_lag_1', 'item_cnt_block_mean_lag_1',
       'shop_cnt_block_lag_1', 'shop_cnt_block_mean_lag_1',
       'category_cnt_block_lag_1', 'category_cnt_block_mean_lag_1',
       'shop_category_cnt_block_lag_1',
       'shop_category_cnt_block_mean_lag_1',
     'comp1_lag_1', 'comp2_lag_1',
       'comp3_lag_1', 'comp4_lag_1', 'comp5_lag_1', 'comp6_lag_1',
       'comp7_lag_1', 'comp8_lag_1', 'comp9_lag_1', 'comp10_lag_1',
       'comp11_lag_1', 'comp12_lag_1', 'comp13_lag_1', 'comp14_lag_1'
    
]

In [47]:
cb_model = CatBoostRegressor(iterations=6000,
                             #learning_rate=0.05, #default is 0.03
                             objective='RMSE',
                             eval_metric='RMSE',
                             #task_type = "GPU",
                             use_best_model=True,
                             early_stopping_rounds = 10,
                             #border_count=32, #number of splits for num features (default 128 on GPU)
                             #bagging_temperature = 20, #default 1 intensity of bootstrap
                             #l2_leaf_reg = 300, #default 3 seems useless
                             #random_strength = 100,#default 1  adds randomness to the split score
                             depth=4,  #default 6
                             #rsm=0.7, #feature %age default 1 no GPU
                             #cat_features=[0,1],
                            #learning_rate=0.001, #default is 0.03
                             random_seed = 42)

#drops = ['subcategory','area']
#x_train = x_train.drop(columns=drops)
#x_val = x_val.drop(columns=drops)


cb_model.fit(x_train[features], y_train, #cat_features=categorical_features_indices,
             eval_set=(x_val[features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

scores = {}
for i,score in enumerate(cb_model.feature_importances_):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

0:	learn: 1.1098968	test: 1.1100548	best: 1.1100548 (0)	total: 572ms	remaining: 57m 13s
1:	learn: 1.0983943	test: 1.1008722	best: 1.1008722 (1)	total: 1.13s	remaining: 56m 25s
2:	learn: 1.0873095	test: 1.0921249	best: 1.0921249 (2)	total: 1.68s	remaining: 55m 50s
3:	learn: 1.0767810	test: 1.0838840	best: 1.0838840 (3)	total: 2.24s	remaining: 56m
4:	learn: 1.0666899	test: 1.0761439	best: 1.0761439 (4)	total: 2.79s	remaining: 55m 48s
5:	learn: 1.0571083	test: 1.0688146	best: 1.0688146 (5)	total: 3.35s	remaining: 55m 44s
6:	learn: 1.0479755	test: 1.0617664	best: 1.0617664 (6)	total: 3.89s	remaining: 55m 28s
7:	learn: 1.0392439	test: 1.0548881	best: 1.0548881 (7)	total: 4.44s	remaining: 55m 27s
8:	learn: 1.0309235	test: 1.0485082	best: 1.0485082 (8)	total: 5s	remaining: 55m 30s
9:	learn: 1.0228425	test: 1.0423063	best: 1.0423063 (9)	total: 5.55s	remaining: 55m 25s
10:	learn: 1.0150766	test: 1.0362269	best: 1.0362269 (10)	total: 6.1s	remaining: 55m 22s
11:	learn: 1.0078499	test: 1.0306565	b

92:	learn: 0.8448474	test: 0.9033420	best: 0.9033420 (92)	total: 53.6s	remaining: 56m 46s
93:	learn: 0.8444213	test: 0.9028896	best: 0.9028896 (93)	total: 54.2s	remaining: 56m 48s
94:	learn: 0.8441296	test: 0.9027493	best: 0.9027493 (94)	total: 54.8s	remaining: 56m 46s
95:	learn: 0.8438119	test: 0.9023961	best: 0.9023961 (95)	total: 55.4s	remaining: 56m 45s
96:	learn: 0.8435294	test: 0.9022724	best: 0.9022724 (96)	total: 56s	remaining: 56m 47s
97:	learn: 0.8432372	test: 0.9020312	best: 0.9020312 (97)	total: 56.5s	remaining: 56m 44s
98:	learn: 0.8428090	test: 0.9015508	best: 0.9015508 (98)	total: 57.1s	remaining: 56m 46s
99:	learn: 0.8424190	test: 0.9010601	best: 0.9010601 (99)	total: 57.8s	remaining: 56m 49s
100:	learn: 0.8420848	test: 0.9007536	best: 0.9007536 (100)	total: 58.4s	remaining: 56m 50s
101:	learn: 0.8417472	test: 0.9004214	best: 0.9004214 (101)	total: 59s	remaining: 56m 51s
102:	learn: 0.8414954	test: 0.9002858	best: 0.9002858 (102)	total: 59.6s	remaining: 56m 52s
103:	lea

181:	learn: 0.8290763	test: 0.8912517	best: 0.8912517 (181)	total: 1m 49s	remaining: 58m 26s
182:	learn: 0.8290091	test: 0.8912441	best: 0.8912441 (182)	total: 1m 50s	remaining: 58m 26s
183:	learn: 0.8289088	test: 0.8910773	best: 0.8910773 (183)	total: 1m 50s	remaining: 58m 26s
184:	learn: 0.8288101	test: 0.8909274	best: 0.8909274 (184)	total: 1m 51s	remaining: 58m 25s
185:	learn: 0.8287190	test: 0.8908282	best: 0.8908282 (185)	total: 1m 52s	remaining: 58m 23s
186:	learn: 0.8286085	test: 0.8906552	best: 0.8906552 (186)	total: 1m 52s	remaining: 58m 22s
187:	learn: 0.8284888	test: 0.8905757	best: 0.8905757 (187)	total: 1m 53s	remaining: 58m 20s
188:	learn: 0.8283969	test: 0.8905672	best: 0.8905672 (188)	total: 1m 53s	remaining: 58m 18s
189:	learn: 0.8282847	test: 0.8904860	best: 0.8904860 (189)	total: 1m 54s	remaining: 58m 15s
190:	learn: 0.8282087	test: 0.8904200	best: 0.8904200 (190)	total: 1m 54s	remaining: 58m 13s
191:	learn: 0.8280552	test: 0.8904379	best: 0.8904200 (190)	total: 1m 

270:	learn: 0.8224937	test: 0.8867726	best: 0.8867726 (270)	total: 2m 41s	remaining: 56m 54s
271:	learn: 0.8224450	test: 0.8867456	best: 0.8867456 (271)	total: 2m 42s	remaining: 56m 52s
272:	learn: 0.8223714	test: 0.8866102	best: 0.8866102 (272)	total: 2m 42s	remaining: 56m 52s
273:	learn: 0.8223031	test: 0.8865997	best: 0.8865997 (273)	total: 2m 43s	remaining: 56m 51s
274:	learn: 0.8222381	test: 0.8864910	best: 0.8864910 (274)	total: 2m 43s	remaining: 56m 50s
275:	learn: 0.8221864	test: 0.8864687	best: 0.8864687 (275)	total: 2m 44s	remaining: 56m 49s
276:	learn: 0.8220718	test: 0.8864789	best: 0.8864687 (275)	total: 2m 45s	remaining: 56m 49s
277:	learn: 0.8219954	test: 0.8863637	best: 0.8863637 (277)	total: 2m 45s	remaining: 56m 49s
278:	learn: 0.8219590	test: 0.8863580	best: 0.8863580 (278)	total: 2m 46s	remaining: 56m 48s
279:	learn: 0.8219193	test: 0.8863428	best: 0.8863428 (279)	total: 2m 46s	remaining: 56m 46s
280:	learn: 0.8218904	test: 0.8863372	best: 0.8863372 (280)	total: 2m 

359:	learn: 0.8182646	test: 0.8838180	best: 0.8838180 (359)	total: 3m 32s	remaining: 55m 28s
360:	learn: 0.8182417	test: 0.8837962	best: 0.8837962 (360)	total: 3m 33s	remaining: 55m 28s
361:	learn: 0.8182237	test: 0.8838113	best: 0.8837962 (360)	total: 3m 33s	remaining: 55m 26s
362:	learn: 0.8181922	test: 0.8838140	best: 0.8837962 (360)	total: 3m 34s	remaining: 55m 25s
363:	learn: 0.8181741	test: 0.8838266	best: 0.8837962 (360)	total: 3m 34s	remaining: 55m 24s
364:	learn: 0.8181512	test: 0.8838278	best: 0.8837962 (360)	total: 3m 35s	remaining: 55m 22s
365:	learn: 0.8181119	test: 0.8838391	best: 0.8837962 (360)	total: 3m 35s	remaining: 55m 22s
366:	learn: 0.8180579	test: 0.8838656	best: 0.8837962 (360)	total: 3m 36s	remaining: 55m 21s
367:	learn: 0.8180281	test: 0.8838401	best: 0.8837962 (360)	total: 3m 37s	remaining: 55m 21s
368:	learn: 0.8179801	test: 0.8838157	best: 0.8837962 (360)	total: 3m 37s	remaining: 55m 23s
369:	learn: 0.8179236	test: 0.8837645	best: 0.8837645 (369)	total: 3m 

448:	learn: 0.8150625	test: 0.8817801	best: 0.8817732 (447)	total: 4m 28s	remaining: 55m 16s
449:	learn: 0.8150380	test: 0.8817708	best: 0.8817708 (449)	total: 4m 28s	remaining: 55m 16s
450:	learn: 0.8150040	test: 0.8817265	best: 0.8817265 (450)	total: 4m 29s	remaining: 55m 15s
451:	learn: 0.8149831	test: 0.8817774	best: 0.8817265 (450)	total: 4m 30s	remaining: 55m 14s
452:	learn: 0.8149605	test: 0.8817599	best: 0.8817265 (450)	total: 4m 30s	remaining: 55m 14s
453:	learn: 0.8149328	test: 0.8817244	best: 0.8817244 (453)	total: 4m 31s	remaining: 55m 15s
454:	learn: 0.8149034	test: 0.8816830	best: 0.8816830 (454)	total: 4m 32s	remaining: 55m 16s
455:	learn: 0.8148835	test: 0.8816645	best: 0.8816645 (455)	total: 4m 32s	remaining: 55m 15s
456:	learn: 0.8148619	test: 0.8816500	best: 0.8816500 (456)	total: 4m 33s	remaining: 55m 13s
457:	learn: 0.8147723	test: 0.8816374	best: 0.8816374 (457)	total: 4m 33s	remaining: 55m 15s
458:	learn: 0.8147453	test: 0.8816616	best: 0.8816374 (457)	total: 4m 

537:	learn: 0.8123576	test: 0.8800687	best: 0.8800479 (535)	total: 5m 21s	remaining: 54m 23s
538:	learn: 0.8123300	test: 0.8800044	best: 0.8800044 (538)	total: 5m 22s	remaining: 54m 22s
539:	learn: 0.8122972	test: 0.8799983	best: 0.8799983 (539)	total: 5m 22s	remaining: 54m 21s
540:	learn: 0.8122750	test: 0.8799997	best: 0.8799983 (539)	total: 5m 23s	remaining: 54m 20s
541:	learn: 0.8122571	test: 0.8799963	best: 0.8799963 (541)	total: 5m 23s	remaining: 54m 20s
542:	learn: 0.8122280	test: 0.8799564	best: 0.8799564 (542)	total: 5m 24s	remaining: 54m 19s
543:	learn: 0.8122156	test: 0.8799547	best: 0.8799547 (543)	total: 5m 24s	remaining: 54m 18s
544:	learn: 0.8121924	test: 0.8799174	best: 0.8799174 (544)	total: 5m 25s	remaining: 54m 17s
545:	learn: 0.8121770	test: 0.8799041	best: 0.8799041 (545)	total: 5m 26s	remaining: 54m 17s
546:	learn: 0.8121464	test: 0.8798926	best: 0.8798926 (546)	total: 5m 26s	remaining: 54m 15s
547:	learn: 0.8121084	test: 0.8798655	best: 0.8798655 (547)	total: 5m 

626:	learn: 0.8103764	test: 0.8788484	best: 0.8788484 (626)	total: 6m 15s	remaining: 53m 42s
627:	learn: 0.8103580	test: 0.8788373	best: 0.8788373 (627)	total: 6m 16s	remaining: 53m 41s
628:	learn: 0.8103370	test: 0.8788343	best: 0.8788343 (628)	total: 6m 17s	remaining: 53m 40s
629:	learn: 0.8103228	test: 0.8788155	best: 0.8788155 (629)	total: 6m 17s	remaining: 53m 39s
630:	learn: 0.8103047	test: 0.8788080	best: 0.8788080 (630)	total: 6m 18s	remaining: 53m 38s
631:	learn: 0.8102857	test: 0.8787719	best: 0.8787719 (631)	total: 6m 18s	remaining: 53m 37s
632:	learn: 0.8102731	test: 0.8787710	best: 0.8787710 (632)	total: 6m 19s	remaining: 53m 36s
633:	learn: 0.8102578	test: 0.8787757	best: 0.8787710 (632)	total: 6m 19s	remaining: 53m 35s
634:	learn: 0.8102475	test: 0.8787753	best: 0.8787710 (632)	total: 6m 20s	remaining: 53m 33s
635:	learn: 0.8102262	test: 0.8787611	best: 0.8787611 (635)	total: 6m 20s	remaining: 53m 33s
636:	learn: 0.8101933	test: 0.8787681	best: 0.8787611 (635)	total: 6m 

[('comp6_lag_1', 28.344746200470539),
 ('comp5_lag_1', 27.900905215810653),
 ('comp3_lag_1', 12.100108622761748),
 ('category_cnt_block_lag_1', 9.8462734440401682),
 ('comp4_lag_1', 6.4718296505613031),
 ('comp10_lag_1', 5.2709023774177446),
 ('shop_category_cnt_block_mean_lag_1', 4.1043816129521824),
 ('comp8_lag_1', 3.4610980303396333),
 ('item_cnt_block_lag_1', 2.4997548456460499)]

In [45]:
scores = {}
for i,score in enumerate(cb_model.feature_importances_):
    scores[features[i]] = score

sorted(scores.items(), key=lambda x: x[1])[::-1]

[('comp5_lag_1', 29.619523468178443),
 ('comp6_lag_1', 28.767242053696059),
 ('comp3_lag_1', 10.694369206759404),
 ('category_cnt_block_lag_1', 7.5398353444334543),
 ('comp4_lag_1', 6.1875160600191084),
 ('comp10_lag_1', 5.7385007514986377),
 ('comp8_lag_1', 3.2568575939174953),
 ('shop_category_cnt_block_mean_lag_1', 1.695421375330294),
 ('item_cnt_block_lag_1', 1.3407266273826097),
 ('comp2_lag_1', 0.95358396276977153),
 ('comp7_lag_1', 0.86253303345995891),
 ('comp13_lag_1', 0.85418868550302895),
 ('shop_category_cnt_block_lag_1', 0.59448937296713089),
 ('shop_cnt_block_mean_lag_1', 0.44480510411734114),
 ('item_cnt_block_mean_lag_1', 0.39248657468157372),
 ('comp9_lag_1', 0.28222936336462817),
 ('comp14_lag_1', 0.24222376947996366),
 ('comp11_lag_1', 0.17742350530814605),
 ('comp12_lag_1', 0.12628838114818838),
 ('comp1_lag_1', 0.126284952374773),
 ('category_cnt_block_mean_lag_1', 0.10347081361004445),
 ('shop_cnt_block_lag_1', 0.0)]

In [46]:
features = [item[0] for item in scores.items() if item[1] > 1]

In [75]:
features

['item_cnt_block_lag_1',
 'item_cnt_block_mean_lag_1',
 'category_cnt_block_lag_1',
 'comp3_lag_1',
 'comp4_lag_1',
 'comp5_lag_1',
 'comp6_lag_1',
 'comp8_lag_1',
 'comp10_lag_1',
 'comp14_lag_1']

In [76]:
test            = pd.read_csv('test.csv.gz')
test = test.set_index('item_id').join(items.set_index('item_id'))
test.reset_index(inplace=True)

In [29]:
train = training[training['date_block_num'] == 33]

In [77]:
cols = ['shop_id','item_id']

test = test.merge(train, on=cols,how='left')

In [43]:
len(test)

214200

In [106]:
features = ['item_cnt_block_lag_1',
 'item_cnt_block_mean_lag_1',
 'category_cnt_block_lag_1',
 'comp3_lag_1',
 'comp4_lag_1',
 'comp5_lag_1',
 'comp6_lag_1',
 'comp8_lag_1',
 'comp10_lag_1',
 'comp14_lag_1']

In [79]:

test.drop(columns=['item_cnt_block_lag_1'],inplace=True)
test.rename(columns={"item_cnt_block":"item_cnt_block_lag_1"},inplace=True)

test.drop(columns=['item_cnt_block_mean_lag_1'],inplace=True)
test.rename(columns={"item_cnt_block_mean":"item_cnt_block_mean_lag_1"},inplace=True)

test.drop(columns=['category_cnt_block_lag_1'],inplace=True)
test.rename(columns={"category_cnt_block":"category_cnt_block_lag_1"},inplace=True)

test.drop(columns=['comp3_lag_1'],inplace=True)
test.rename(columns={"comp3":"comp3_lag_1"},inplace=True)

test.drop(columns=['comp4_lag_1'],inplace=True)
test.rename(columns={"comp4":"comp4_lag_1"},inplace=True)

test.drop(columns=['comp5_lag_1'],inplace=True)
test.rename(columns={"comp5":"comp5_lag_1"},inplace=True)

test.drop(columns=['comp6_lag_1'],inplace=True)
test.rename(columns={"comp6":"comp6_lag_1"},inplace=True)

test.drop(columns=['comp8_lag_1'],inplace=True)
test.rename(columns={"comp8":"comp8_lag_1"},inplace=True)

test.drop(columns=['comp10_lag_1'],inplace=True)
test.rename(columns={"comp10":"comp10_lag_1"},inplace=True)

test.drop(columns=['comp14_lag_1'],inplace=True)
test.rename(columns={"comp14":"comp14_lag_1"},inplace=True)

In [101]:
preds = model_lgb.predict(test[features])
preds.clip(0,20,out=preds)

array([ 0.13267908,  0.12405295,  0.16553087, ...,  0.21863482,
        0.25973795,  0.25021062])

In [105]:

print(np.mean(preds))
print(np.max(preds))

submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

0.323770663293
17.2484104745


In [ ]:
bestpreds = pd.read_csv('submissionbest.csv')['item_cnt_month']
print(np.mean(bestpreds))
print(np.max(bestpreds))

In [104]:
lgbm_preds = pd.read_csv('lgbm102.csv')['item_cnt_month']
lstm_preds = pd.read_csv('lstm104.csv')['item_cnt_month']
lr_preds = pd.read_csv('lr111.csv')['item_cnt_month']
cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']



#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

#preds = (lstm_preds * 0.2) + (lgbm_preds * 0.6) + (lr_preds*0.2) #this gives 1.08
#preds = (lgbm_preds * 0.8) + (lr_preds*0.2) # this gives 1.04
preds = (cb_preds * 0.5) + (lgbm_preds*0.5) # this gave 0.98 :)))))))